# Scrapy

## 初识Scrapy

1. 爬虫之前首先要明确你的目的，比如你想通过爬取网页获得哪些数据。明确目标之后就可以分析网页结构及网页数据存储方式。

2. 接下来就可以在Scrapy中编写一个爬虫，即实现一个scrapy.Spider子类

## 编写Spider

先从Scrapy爬虫程序中最核心的组件Spider将起

Spider主要负责提取页面中的数据，并产生对新页面的下载请求

### Scrapy工作流程

![pipeling](scrapy.png)

1. 当SPIDER要爬取某URL地址的页面时,需使用该URL构造一个Request对象,提交给ENGINE(图2-1中的1)

2. Request对象随后进入SCHEDULER按某种算法进行排队,之后的某个时刻SCHEDULER将其出队,送往DOWNLOADER(图2-1中的2、3、4)

3. DOWNLOADER根据Request对象中的URL地址发送一次HTTP请求到网站服务器,之后用服务器返回的HTTP响应构造出一个Response对象,其中包含页面的HTML文本(图2-1中的5)

4. Response对象最终会被递送给SPIDER的页面解析函数(构造Request对象时指定)进行处理,页面解析函数从页面中提取数据,封装成Item后提交给ENGINE,Item之后被送往ITEMPIPELINES进行处理,最终可能由EXPORTER(图2-1中没有显示)以某种数据格式写入文件(csv,json);另一方面,页面解析函数还从页面中提取链接(URL),构造出新的Request对象提交给ENGINE(图2-1中的6、7、8)

### Request和Response对象

如果把框架中的组件比作人体的各个器官,Request和Response对
象便是血液,Item则是代谢产物

#### Request对象

语法:
Request(url[, callback, method='GET', headers, body, cookies, meta,encoding='utf-8', priority=0, dont_filter=False,

*url* 必选

*callback*　指定页面解析函数，Request对象请求的页面下载完成后，由该参数指定的页面解析函数被调用，如果未传递该参数，默认调用Spider的parse方法

*cookies* 传入字典类型的cookies

*meta* Request的元数据字典，字典类型，用于给框架中其他组件传递信息，比如中间件Item Pipeline,其他组件可以使用Request对象的meta属性访问该元数据字典

*priority* 请求的优先级默认值为0,优先级高的请求优先下载

*dont_filter* 默认情况下，对同一url地址多次提交下载请求，后面的请求会被去重过滤器过滤(避免重复下载)。如果将该参数设置为True,可以使得请求避免被过滤，强制下载

*errback* 请求出现异常或者出现HTTP错误时的回调函数

*一般情况下，我们只需要传入url和callback两个参数即可，其他使用默认值即可*

##### meta详解

meta的功能就是从request携带信息，将其传递给response,结构与字典一样

除了可以自定义key,scrapy有一些内置的特殊key:

1. proxy 设置代理

2. download_timeout

3. max_retry_times 最大重试次数，除去第一次，默认2次

4. dont_redirect 设定为True后，Request将不会重定向

5. dont_retry 设定为True后，对于http链接错误或超时的请求将不会重试

6. handle_httpstatus_list http返回码200-300之间都是成功的返回，超出这个范围都是失败返回，scrapy默认是过滤这些返回，不会接收这些错误的返回进行处理。不过这个参数可以自定义处理哪些错误返回


dont_redirect：如果 Request.meta 包含 dont_redirect 键，则该request将会被RedirectMiddleware忽略

dont_retry：如果 Request.meta 包含 dont_retry 键， 该request将会被RetryMiddleware忽略

handle_httpstatus_list：Request.meta 中的 handle_httpstatus_list 键可以用来指定每个request所允许的response code

handle_httpstatus_all：handle_httpstatus_all为True ，可以允许请求的任何响应代码

dont_merge_cookies：Request.meta 中的dont_merge_cookies设为TRUE，可以避免与现有cookie合并

cookiejar：Scrapy通过使用 Request.meta中的cookiejar 来支持单spider追踪多cookie session。 默认情况下其使用一个cookie 

jar(session)，不过可以传递一个标示符来使用多个

dont_cache：可以避免使用dont_cache元键等于True缓存每个策略的响应

redirect_urls：通过该中间件的(被重定向的)request的url可以通过 Request.meta 的 redirect_urls 键找到(**在多次请求被重定向时有用获取被重定向的url然后使用中间件重新请求**)这个值对应的是一个列表。请求自动跳转了几次，这个列表里面就有几个URL

bindaddress：用于执行请求的传出IP地址的IP

dont_obey_robotstxt：如果Request.meta将dont_obey_robotstxt键设置为True，则即使启用ROBOTSTXT_OBEY，RobotsTxtMiddleware也
会忽略该请求

download_timeout：下载器在超时之前等待的时间（以秒为单位）

download_maxsize：爬取URL的最大长度

download_latency：自请求已经开始，即通过网络发送的HTTP消息，用于获取响应的时间量 
该元密钥仅在下载响应时才可用。虽然大多数其他元键用于控制Scrapy行为，但是这个应用程序应该是只读的

download_fail_on_dataloss：是否在故障响应失败

proxy：可以将代理每个请求设置为像http：// some_proxy_server：port这样的值

ftp_user ：用于FTP连接的用户名

ftp_password ：用于FTP连接的密码

referrer_policy：为每个请求设置referrer_policy

max_retry_times：用于每个请求的重试次数。初始化时，max_retry_times元键比RETRY_TIMES设置更高优先级

In [ ]:
yield scrapy.Request(url= 'https://httpbin.org/get/zarten', 
                     meta= {'handle_httpstatus_list' : [404]})

7.　handle_httpstatus_all 设定为True后，Response将接收处理任意状态码的返回信息

8. dont_merge_cookies scrapy会自动保存返回的cookies,用于它的下次请求，当我们指定了自定义cookies时，我们不需要合并的返回的cookies而使用自己指定的cookies,可以设定为True



In [ ]:
def parse_page1(self, response):
    item = MyItem()
    item['main_url'] = response.url
    request = scrapy.Request("http://www.example.com/some_page.html",
                             callback=self.parse_page2)
    request.meta['item'] = item　# request.meta()存入元数据
    yield request

def parse_page2(self, response):
    item = response.meta['item']　# response.meta()获取request的信息
    item['other_url'] = response.url
    yield item

In [3]:
'''
import scrapy.Request
import scrpay

request = scrapy.Request(url='http://httpbin.org/post',
                 method='post',
                 callback=parse)

def parse(response):
    pass

'''

"\nimport scrapy.Request\nimport scrpay\n\nrequest = scrapy.Request(url='http://httpbin.org/post',\n                 method='post',\n                 callback=parse)\n\ndef parse(response):\n    pass\n\n"

#### Response对象

Response对象用来描述一个HTTP响应，Response只是一个基类，根据响应内容的不同有如下子类
1. TextResponse
2. HtmlResponse
3. XmlResponse

当一个页面下载完成时，下载器依据HTTP响应头部中的Content-Type信息创建某个Response的子类对象

以HtmlResponse子类介绍其属性和方法

1. url:HTTP响应的url地址，str类型

2. status　响应状态码

3. headers:HTTP响应头信息，类字典类型可以调用get方法获取信息

4. body: HTTP响应正文，bytes类型

5. text: 文本形式的HTTP响应正文，str类型，由response,bodys使用response.encoding解码得到的

6. request:产生该HTTP响应的Request对象

7. meta:即response.request.meta,在构建Request对象时，可将要传递给响应处理函数的信息通过meta参数传入。响应处理函数处理响应是，通过response.meta将信息取出

8. selector:Selector对象用于在Response中提取数据

9. xpath: 使用XPath选择器在Response中提取数据，实际上是response.selector.xpath方法的快捷方式

10. css:　使用CSS选择器在Response中提取数据，实际上是response.selector.css方法的快捷方式

11. urljoin: 用于构造绝对url,当传入的url参数是一个相对地址时，根据response.url计算出响应的绝对url

*虽然HtmlResponse对象有很多属性，但最常用的是xpath,css,urljoin,前两个方法用于提取数据，后一个方法用于构造绝对url*

##### response.request

获取此response的请求体request信息,如同浏览器返回的一样

如response.request.headers['User-Agent']获取UA信息

### Spider开发流程

实现Spider子类要有这样的逻辑

1. 爬虫从哪个或哪些页面开始爬取?

2. 对于一个已下载的页面,提取其中的哪些数据?

3. 爬取完当前页面后,接下来爬取哪个或哪些页面?

In [ ]:
import scrapy


class BookSpider(scrapy.Spider):
    name = 'book'
    allowed_domains = ['books.toscrape.com']
    start_urls = ['http://books.toscrape.com']

    def parse(self, response):
        #print(response.status)
        item = BookToscrapeItem()
        for i in response.css('.row > li'):
            item['bk_name'] = i.css('h3 a::attr(title)').extract_first()
            #print(bk_name)
            item['price'] = i.css('.price_color::text').extract_first()
            yield item

        nextpage = response.css('.next a::attr(href)').extract_first()
        if next_page:
            next_url = response.urljoin(nextpage)
            yield scrapy.Request(next_url, callback=self.parse)

实现一个Spider只需要下面4个步骤:

1. 继承scarpy.Spider
2. 为Spider取名
3. 设定起始爬取点
4. 实现页面解析函数

#### 继承scrapy.Spider

Scrapy框架提供了一个Spider基类，编写的Spider需要继承它

#### 为Spider命名

在一个Scrapy项目中可以实现多个Spider,每个Spider需要有一个能够区分彼此的唯一标识，Spider的类属性name便是这个唯一标识

#### 设定起始爬取点

Spider必然要从某个或某些页面开始爬取,我们称这些页面为
起始爬取点,可以通过类属性start_urls来设定起始爬取点

如果构建Spider时没有调用scrapy.Request方法，那么会自动调用Spider基类中的start_requests方法

起始爬取点可能有多个，start_requests方法需要返回一个可迭代对象，其中每一个元素是一个Request对象

由于起始爬取点的下载请求是有引擎调用Spider对象的start_requests方法产生的，因此我们也可以在自定义的Spider中实现start_requests方法(覆盖基类Spider的start_requests方法)，
直接构造并提交起始爬取点的Request对象。

在某些场景下使用这种
方式更加灵活,例如有时想为Request添加特定的HTTP请求头部,
或想为Request指定特定的页面解析函数

In [ ]:
class BookSpider(scrapy.Spider):
    name = 'book'
    allowed_domains = ['books.toscrape.com']
    start_urls = ['http://books.toscrape.com']

    def start_requests(self):
        # 覆盖Spider基类中的方法
        yield scrapy.Request(url='http://books.toscrape.com',
                             method='get',
                             callback=self.parse_book)

    def parse_book(self,response):
        #print(response.status)
        item = BookToscrapeItem()
        for i in response.css('.row > li'):
            item['bk_name'] = i.css('h3 a::attr(title)').extract_first()
            #print(bk_name)
            item['price'] = i.css('.price_color::text').extract_first()
            yield item

        nextpage = response.css('.next a::attr(href)').extract_first()
        if nextpage:
            next_url = response.urljoin(nextpage)
            yield scrapy.Request(next_url, callback=self.parse_book)

#### 实现页面解析函数

页面解析函数也就是构造Request对象通过callback参数指定的回调函数（或默认的parse方法）。页面解析函数是实现SPider中最核心的部分，它需要完成以下两项工作:

1. 使用选择器爬取页面中的数据，将数据封装后(Item或字典)提交给Scrapy引擎

2. 使用选择器提取页面中的链接，用其构造新的Request对象并提交给Scrapy引擎(下载链接页面)

一个页面中可能包含多项数据以及多个链接，因此页面解析函数被要求返回一个可迭代对象(通常被实现成一个生成器函数),每次返回一项数据(Item或字典)或一个Request对象

# 使用Selector提取数据

从页面中提取数据的核心技术是HTTP文本解析，在Python中常用以下模块处理此类问题:

1. BeautifulSoup
API简洁易用，但解析速度较慢

2. lxml
是一套由C语言编写的xml解析库(libxml2),解析速度更快，API相对复杂

Scrapy综合上述两者优点实现了Selector类,它是基于lxml库
构建的,并简化了API接口。在Scrapy中使用Selector对象提取页面
中的数据,使用时先通过XPath或CSS选择器选中页面中要提取的数
据,然后进行提取

## Selector对象

### 创建对象

Selector类的实现位于scrapy.selector模块，创建Selector对象时，可将页面的HTML文档字符串传递给Selector构造器方法的text参数

In [4]:
from scrapy.selector import Selector

text = '''
<html>
    <body>
        <h1>Hello World</h1>
        <h1>Hello Scrapy</h1>
        <b>Hello python</b>
        <ul>
            <li>C++</li>
            <li>Java</li>
            <li>Python</li> 
        </ul> 
    </body>
</html>
'''

selector = Selector(text=text)

selector

<Selector xpath=None data='<html>\n    <body>\n        <h1>Hello W...'>

也可以使用一个Response对象构造Selector对象,将其传递给
Selector构造器方法的response参数

In [10]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from scrapy.http import TextResponse
body = '''
<html>
    <body>
        <h1>Hello World</h1>
        <h1>Hello Scrapy</h1>
        <b>Hello python</b>
        <ul>
            <li>C++</li>
            <li>Java</li>
            <li>Python</li> 
        </ul> 
    </body>
</html>
'''

response = HtmlResponse(url='http://www.example.com',body=body,encoding='utf-8')
selector = Selector(response=response)
selector

<Selector xpath=None data='<html>\n    <body>\n        <h1>Hello W...'>

### 选中对象

通过Selctor对象中的xpath方法或css方法，选中文档中的某个或某些部分

xpath和css方法返回一个SelectorList对象，其中包含每个被选中部分的Selector对象，SelectorList支持列表接口，可使用for语句迭代访问其中的每一个Selector对象


SelectorList对象也有xpath和css方法,因此可以提取嵌套数据

In [15]:
selector_list = selector.xpath('//h1')
selector_list

for item in selector_list:
    print(item)
    
selector_list.xpath('./text()')

<Selector xpath='//h1' data='<h1>Hello World</h1>'>
<Selector xpath='//h1' data='<h1>Hello Scrapy</h1>'>


[<Selector xpath='./text()' data='Hello World'>,
 <Selector xpath='./text()' data='Hello Scrapy'>]

### 提取数据

调用Selector或SelectorList对象的以下方法可将选中的内容提取:

1. extract()

2. re()

3. extract_first() (SelectorList专有)

4. re_first()　　　　(SelectorList专有)

#### extract()

调用selector对象的extract方法将返回选中内容的Unicode字符串

In [18]:
s1 = selector.xpath('.//li')
s1[0].extract()

'<li>C++</li>'

SelectorList对象
的extract方法内部会调用其中每个Selector对象的extract方法,并
把所有结果收集到一个列表返回给用户

In [19]:
s1.extract()

['<li>C++</li>', '<li>Java</li>', '<li>Python</li>']

#### extract_first()

SelectorList对象还有一个extract_first方法,该方法返回其中
第一个Selector对象调用extract方法的结果

返回的是**字符串**而不是列表

#### re()

有些时候我们想使用正则表达式提取选中的内容，可以使用re方法

In [26]:
text = '''
<ul>
    <li>Python 学习手册 <b>价格: 99.00 元</b></li>
    <li>Python 核心编程 <b>价格: 88.00 元</b></li>
    <li>Python 基础教程 <b>价格: 80.00 元</b></li>
</ul>
'''

selector = Selector(text=text)

selector.xpath('.//li/b/text()').re(r'\d+.\d+')

['99.00', '88.00', '80.00']

#### re_first()

和extract_first效果一样

In [27]:
selector.xpath('.//li/b/text()').re_first(r'\d+.\d+')

'99.00'

## Response内置Selector

在实际开发中,几乎不需要手动创建Selector对象,在第一次访
问一个Response对象的selector属性时,Response对象内部会以自
身为参数自动创建Selector对象,并将该Selector对象缓存,以便下
次使用

In [ ]:
''''
class TextResponse(Response):
def __init__(self, *args, **kwargs):
    ...
    self._cached_selector = None
    ...
    @property
    def selector(self):
    from scrapy.selector import Selector
    if self._cached_selector is None:
    self._cached_selector = Selector(self)
    return self._cached_selector
    ...
    
'''

In [34]:
class test:
    def __init__(self,*args,**kwargs):
        
        self._val = None
    @property
    def my(self):
        self._val = 12
        return self._val


12

In [5]:
from scrapy.http import HtmlResponse

body = '''
<html>
    <body>
        <h1>Hello World</h1>
        <h1>Hello Scrapy</h1>
        <b>Hello python</b>
        <ul>
            <li>C++</li>
            <li>Java</li>
            <li>Python</li> 
        </ul> 
    </body>
</html>
'''

response = HtmlResponse(url='http://www.example.com',body=body,encoding='utf-8')

response.selector

<Selector xpath=None data='<html>\n    <body>\n        <h1>Hello W...'>

# 使用Item封装数据

应该用怎样的数据结构维护零散的信息字段，在Scrapy中可以使用自定义的Item类封装爬取到的数据

## Item和Field

Scrapy提供了以下两个类，用户可以使用它们自定义数据类，封装爬取到的数据

1. Item基类

2. Field类

自定义一个数据类，只需要继承Item,并创建一系列Field对象的类属性

In [12]:
from scrapy import Item,Field

class BookItem(Item):
    # 定义两个字段
    name = Field()
    price = Field()

book1 = BookItem(name='three mao',price=45)

# Item支持字典接口,Item在使用上和Python字典类似
book1['name']

book2 = BookItem()
book2['name'] = 'hello world'

# 对字段赋值时，若赋值给没有定义的字段，会抛出异常
book2['sex'] = 'boy'

KeyError: 'BookItem does not support field: sex'

## 拓展Item子类

有些时候，我们可能根据需求对已有的自定义数据类进行拓展，例如又新建了一个Spider，负责在爬取国外书籍(中文翻译版)的信息，此类书籍的信息比之前多了一个译者字段，此时可以继承之前定义的BookItem定义一个新类

In [13]:
class BookItem(Item):
    # 定义两个字段
    name = Field()
    price = Field()

class foreignBookItem(BookItem):
    translator =  Field()
    
book = foreignBookItem()
book['translator'] = 'chenzhi'

## Field元数据

元数据是用来描述数据的数据（Data that describes other data），在数据库中元数据即字段名

一项数据由Spider提交给Scrapy引擎后，可能会被递送给其他组件(Item Pipeling、Exporter)处理。假设想传递额外信息给处理数据的某个组件(例如，告诉该组件应以怎样的方式处理数据)，此时可以使用Field元数据

实际上,Field是python字典的子类，可以通过键获取Field对象中的元数据

In [16]:
class ExampleItem(Item):
    x = Field(a='hello', b=[1, 2, 3])
    y = Field(a=lambda x: x ** 2)
    
e = ExampleItem(x=100,y=200)

# fields属性得到一个包含所有Field字典的对象
print(e.fields)

# Field是字典的子类
print(issubclass(Field,dict))

{'x': {'a': 'hello', 'b': [1, 2, 3]}, 'y': {'a': <function ExampleItem.<lambda> at 0x7f68bc3739e0>}}
True


In [18]:
# 获取Field对象中的元数据
filed_x = e.fields['x']
filed_x

{'a': 'hello', 'b': [1, 2, 3]}

# 使用Item Pipeline处理数据

在Scrapy中，Item Pipeline是处理数据的组件，一个Item Pipeline是处理数据的组件，一个Item Pipeline就是一个包含特定接口的类，通常只负责一种功能的数据处理，在一个项目中可以同时启用多个Item Pipeline,它们按指定次序级联起来，形成一条数据处理流水线

Item Pipeline典型应用:

1. 清洗数据
2. 验证数据的有效性
3. 过滤掉重复的数据
4. 将数据存入数据库

一个Item Pipeline不需要继承特定基类，只需要实现某些特定方法，例如process_item、open_spider、close_spider

一个Item Pipeline必须实现一个process_item(item,spider)方法，该方法用来处理每一项由Spider爬取到的数据，其中的两个参数:

1. item 爬取到的一项数据(Item或字典)
2. Spider 爬取此数据的Spider对象

process_item是Item Pipeline的核心，如果process_item在处理某项item时返回了一项数据(item或字典)，返回的数据会递送给下一级Item pipeline(如果有)继续处理

如果process_item在处理某项item时抛出一个DropItem异常，该项item便会被抛弃，不在递送给后面的Item Pipeline继续处理，也不会导出到文件

除了实现process_item方法外，还有3个常用的方法:

1. open_spider(self,spider)

Spider打开时(处理数据前)回调该方法，通常在开始处理数据之前完成某些初始化工作，如连接数据库

2. close_spider(self,spider)

Spider关闭时(处理数据后)回调该方法，通常该方法用于在处理完所有数据之后完成某些清理工作，如关闭数据库

3. from_crawler(cls,crawler)

创建Item Pipeline对象时回调该方法。通常在该方法中通过crawel.settings读取配置，根据配置创建Item Pipeline对象

## 启用Item Pipeline

在Scrapy中，Item Pipeline是可选的组件，想要启用某个(或某些)Item Pipeline,需要在配置文件settings.py中进行配置

### 过滤重复数据

In [ ]:
from scrapy.exceptions import DropItem

class DuplicatesPipeline(object):
    def __init__(self):
        self.book_set = set()
    def process_item(self, item, spider):
        name = item['name']
        if name in self.book_set:
            raise DropItem("Duplicate book found: %s" % item)
        
        self.book_set.add(name)
        return item

### 将数据存入MongoDB

insert_one()方法需要传入一个字典对象,不能传入Item对象

In [ ]:
from scrapy.item import Item
import pymongo

class MongoDBPipeline(object):
    # 定义两个常量
    DB_URI = 'mongodb://localhost:27017/'
    DB_NAME = 'scrapy_data'
    def open_spider(self, spider):
        self.client = pymongo.MongoClient(self.DB_URI)
        self.db = self.client[self.DB_NAME]
    def close_spider(self, spider):
        self.client.close()
    def process_item(self, item, spider):
        collection = self.db[spider.name]
        post = dict(item) if isinstance(item, Item) else item
        collection.insert_one(post)
        return item

上述代码中，数据库的URL地址和数据库的名字硬编码在代码中，如果希望通过配置文件设置它们，只需稍作改动

In [ ]:
class MongoDBPipeline(object):
    
    # 增加类方法，替代在类属性中定义DB.URL和DB_name
    @classmethod
    def from_crawler(cls, crawler):
        cls.DB_URI = crawler.settings.get('MONGO_DB_URI',
        'mongodb://localhost:27017/')
        cls.DB_NAME = crawler.settings.get('MONGO_DB_NAME', 'scrapy_d
        return cls()
    def open_spider(self, spider):
        self.client = pymongo.MongoClient(self.DB_URI)
        self.db = self.client[self.DB_NAME]
    def close_spider(self, spider):
        self.client.close()
    def process_item(self, item, spider):
        collection = self.db[spider.name]
        post = dict(item) if isinstance(item, Item) else item
        collection.insert_one(post)
        return item

如果一个Item Pipeline定义了from_crawler方法，Scrapy就会调用该方法来创建Item Pipeline对象

该方法有两个参数:

1. cls Item Pipeline类的对象
2. crawler　Crawler是Scrapy中的一个核心对象，可以通过crawler的setting属性访问配置文件

# 使用LinkExtractor提取链接

在爬取一个网站时，想要爬取的数据通常分布在多个页面中，每个页面包含一部分数据以及到其他页面的链接，提取链接有使用Selector和LinkExtractor两种方法

1. 因为链接也是页面中的数据，所以可以使用与提取数据相同的方法进行提取，在提取少量(几个)链接或提取规则比较简单时，使用Selector就足够了

2. 使用LinkExtractor

Scrapy提供了一个专门用于提取链接的类LinkExtractor,在提取大量链接或提取规则比较复杂时，使用LinkExtractor更加方便

In [ ]:
def parse_book(self,response):
    #print(response.status)
    item = BookToscrapeItem()
    for i in response.css('.row > li'):
        item['bk_name'] = i.css('h3 a::attr(title)').extract_first()
        #print(bk_name)
        item['price'] = i.css('.price_color::text').extract_first()
        yield item

    nextpage = response.css('.next a::attr(href)').extract_first()
    if nextpage:
        # 如果找到下一页的url,得到绝对路径,构造新的Request 对象
        next_url = response.urljoin(nextpage)
        yield scrapy.Request(next_url, callback=self.parse_book)

使用LinkExtractor非常简单

1. 导入LinkExtractor，位于scrapy.linkextractors模块

2. 创建一个LinkExtractor对象，**默认提取页面中的所有链接**，也可以使用一个或多个构造器参数描述提取规则，传递给restrict_css参数一个CSS选择器表达式

3. 调用LinkExtractor对象的extract_links方法传入一个Response对象所包含的页面中提取链接，最终返回一个列表。其中每一个元素都是一个Link对象，即提取到的一个链接

4. Link对象的url属性便是链接页面的绝对url地址(无须调用response.urljoin方法)，用其构造Request对象并提交

In [ ]:
from scrapy.linkextractors import LinkExtractor
class BooksSpider(scrapy.Spider):
...
def parse(self, response):
...
    # 提取链接
    # 下一页的url 在ul.pager > li.next > a 里面
    # 例如: <li class="next"><a href="catalogue/page-2.html">next
    le = LinkExtractor(restrict_css='ul.pager li.next')
    links = le.extract_links(response)
    if links:
        next_url = links[0].url
        yield scrapy.Request(next_url, callback=self.parse)

In [19]:
from scrapy.http import HtmlResponse

html1 = open('example1.html').read()
html2 = open('example2.html').read()

response1 = HtmlResponse(url='http://example1.com',
                        body=html1,
                        encoding='utf-8')
response2 = HtmlResponse(url='http://example2.com',
                        body=html2,
                        encoding='utf-8')



In [21]:
from scrapy.linkextractors import LinkExtractor

le = LinkExtractor()

print(le.extract_links(response1))

[Link(url='http://example1.com/intro/install.html', text='Installatio\n            ', fragment='', nofollow=False), Link(url='http://example1.com/intro/tutorial.html', text='Tutorial\n            ', fragment='', nofollow=False), Link(url='http://example1.com/examples.html', text='Examples', fragment='', nofollow=False), Link(url='http://stackoverflow.com/tags/scrapy/info', text='StackO\n            ', fragment='', nofollow=False), Link(url='https://github.com/scrapy/scrapy', text='Fork on Github<\n        ', fragment='', nofollow=False)]


LinkExtractor构造器的各个参数:

1. allow

接受一个正则表达式或一个正则表达式列表，提取绝对url与正则表达式匹配的链接，如果该参数为空(默认)，就提取全部链接

In [23]:
# 提取页面example1.html中路径以/intro开始的链接

pattern = '/intro/.+\.html$'

le = LinkExtractor(allow=pattern)

links = le.extract_links(response1)

[link.url for link in links]

['http://example1.com/intro/install.html',
 'http://example1.com/intro/tutorial.html']

2. deny

接收一个正则表达式或一个正则表达式列表，与allow相反，排除绝对url与正则表达式匹配的链接

In [30]:
# 提取页面example1.html中所有站外链接

pattern1 = '^'+response1.url

le = LinkExtractor(deny=pattern1)
links = le.extract_links(response1)
print(links)

[Link(url='http://stackoverflow.com/tags/scrapy/info', text='StackO\n            ', fragment='', nofollow=False), Link(url='https://github.com/scrapy/scrapy', text='Fork on Github<\n        ', fragment='', nofollow=False)]


3. allow_domains

接收一个域名或一个域名列表，提取指定域的链接

In [31]:
#提取页面example1.html中所有到github.com和stackoverflow.com这两个域的链接

domains = ['github.com','stackoverflow.com']

le = LinkExtractor(allow_domains=domains)

links = le.extract_links(response1)

print(links)

[Link(url='http://stackoverflow.com/tags/scrapy/info', text='StackO\n            ', fragment='', nofollow=False), Link(url='https://github.com/scrapy/scrapy', text='Fork on Github<\n        ', fragment='', nofollow=False)]


4. deny_domains

与allow_domains相反

5. restrict_xpaths

接收一个Xpath表达式或一个Xpath表达式列表，提取Xpath表达式选中区域下的链接

In [33]:
# 提取页面example1.html中<div id="top">元素下的链接

le = LinkExtractor(restrict_xpaths='//div[@id="top"]')

links = le.extract_links(response1)

print(links)

[Link(url='http://example1.com/intro/install.html', text='Installatio\n            ', fragment='', nofollow=False), Link(url='http://example1.com/intro/tutorial.html', text='Tutorial\n            ', fragment='', nofollow=False), Link(url='http://example1.com/examples.html', text='Examples', fragment='', nofollow=False)]


6. restrict_css

基于css选择器选择标签

7. tags

接收一个标签(字符串)或一个标签列表，提取指定标签内的链接，默认为['a','area']

8. attrs

接收一个属性(字符串)或一个属性列表，提取指定属性内的链接，默认为['href']

In [34]:
# 提取页面example2.html中引用JavaScript文件的链接

le = LinkExtractor(tags='script',attrs='src')

links = le.extract_links(response2)

print(links)

[Link(url='http://example2.com/js/app1.js', text='', fragment='', nofollow=False), Link(url='http://example2.com/js/app2.js', text='', fragment='', nofollow=False)]


9. proccess_value

接收一个形如func(value)的回调函数，如果传递了该参数，LinkExtractor将调用该回调函数对提取的每一个链接进行处理，回调函数正常情况下应返回一个字符串,想要抛弃锁处理的链接时，返回None

In [35]:
import re

def process(value):
    m = re.search("javascript:goToPage\('(.*?)'", value)
    
    if m:
        value = m.group(1)
    return value

le = LinkExtractor(process_value=process)

links = le.extract_links(response2)

print(links)

[Link(url='http://example2.com/home.html', text='主页', fragment='', nofollow=False), Link(url='http://example2.com/doc.html', text='文档\n        ', fragment='', nofollow=False), Link(url='http://example2.com/example.html', text='\n', fragment='', nofollow=False)]


# 使用Exporter导出数据

在Scrapy中，负责导出数据的组件被称为Exporter，Scrapy内部实现了多个Exporter,每个Exporter实现一种数据格式的导出

支持的数据格式如下:

1. Json
2. Json lines
3. CSV
4. XML
5. Pickle
6. Marshal

## 指定如何导出数据

在导出数据时，须向Scrapy提供以下信息:

- 导出文件路径

- 导出数据格式(即选用哪个Exporter)

可以通过以下两种方式指定爬虫如何导出数据:

- 通过命令行参数指定

- 通过配置文件指定

### 命令行参数

在运行scrapy crawl 命令时，可以分别指定 **-o** 和 **-t**参数指定导出文件路径及导出数据格式

scrapy crawl books -t csv -o books1.data

指定导出文件路径时,还可以使用%(name)s和%(time)s
两个特殊变量:

- %(name)s: 会被替换为Spider的名字

- %(time)s: 会被替换为文件创建时间

对于任意的一次爬取，都可以使用'export_data/%(name)s/%(time)s.csv',Scrapy爬虫会依据Spider的名字和爬取的时间点创建导出文件

使用命令行参数指定如何导出数据很方便，但命令行参数只能指定导出文件路径以及导出数据格式，并且每次都在命令行里输入很长的参数让人很烦躁，使用配置文件可以弥补这些不足

### 配置文件

#### FEED_URL

导出文件路径

FEED_URI = 'export_data/%(name)s.data'

#### FEED_FORMAT

导出数据格式

FEED_FORMAT = 'csv'

#### FEED_EXPORT_ENCODING

导出文件编码(默认情况下json文件使用数字编码,其他使
用utf-8编码)

FEED_EXPORT_ENCODING = 'gbk'

#### FEED_EXPORT_FIELDS

导出数据包含的字段(默认情况下导出所有字段),并指定
次序

#### FEED_EXPORTERS

用户自定义Exporter字典,添加新的导出数据格式时使用

FEED_EXPORTERS = {'excel': 'my_project.my_exporters.ExcelItemExport}

## 添加导出数据格式

在某些需求下，我们想要添加新的导出数据格式，此时需要实现新的Exporter类

Exporter都是BaseItemExporter的一个子类，BaseItemExporter定义了一些抽象接口待子类实现

- export_item(self,item)

负责导出爬取到的每一项数据，参数item为一项爬取到的数据，每个子类必须实现该方法

- start_exporting(self)

在导出开始时被调用，可在该方法中执行某种某些初始化工作

- finish_exporting(self)

在导出完成时被调用，可在该方法中执行某些清理工作

# 下载文件和图片

下载文件也是实际应用中很常见的一种需求，例如使用爬虫爬取网站中的图片、视频、Word文件、PDF文件、压缩包等

## FilesPipeline和ImagesPipeline

Scrapy框架内部提供了两个Item Pipeline,专门用于下载文件和图片

我们可以将这两个Item Pipeline看作特殊的下载器，用户使用时只需要通过Item的一个特殊字段将要下载文件或图片的url传递给它们，它们会自动将文件或图片下载到本地，并将下载结果信息存入item的另一个字段，以便用户在导出文件中查询

### FilesPipeline使用说明

1. 在配置文件中启用FilesPipeline,通常将其置于其他Item Pipeline之前

2. 在配置文件中，使用FILES_STORE指定文件下载目录

3. 在Spider解析一个包含文件下载链接的页面时，将所有需要下载文件的url地址收集到一个列表，赋给item的file_urls字段。FilePipeline在处理每一项item时，会读取item['file_urls'],对其中每一个url进行下载

当FilesPipeline下载完item['file_urls']中的所有文件后，会将各文件的下载结果信息收集到另一个列表，赋给item的files字段

下载结果信息包括:

- 文件下载到本地的路径
- 文件的校验和
- 文件的url地址

### ImagePipeline使用说明

下载图片本质上也是下载文件，ImagePipeline是FilePipeline的子类，使用上大同小异，只是在所使用的item字段和配置选项上略有差别

item['image_urls']

ImagePipeline在FilePipeline的基础上针对图片增加了一些特有的功能:

- 为图片生成缩略图

开启该功能，只需在配置文件settings.py中设置IMAGES_THUMBS,它是一个字典，每一项的值是缩略图的尺寸

如定义两个缩略图
IMAGES_THUMBS = {
'small': (50, 50),
'big': (270, 270),
}

- 过滤尺寸过小的图片

开启该功能，需在配置文件中设置IMAGES_MIN_WIDTH和IMAGES_MIN_HEIGHT

IMAGES_MIN_WIDTH = 110
IMAGES_MIN_HEIGHT = 110

# 模拟登录

模拟登录时，必须保证settings.py里的COOKIES_ENABLED处于开启状态

## 直接POST数据

FormRequest 是Scrapy发送POST请求的方法，将formdata传递给formdata参数

In [ ]:
import scrapy


class Renren1Spider(scrapy.Spider):
    name = "renren1"
    allowed_domains = ["renren.com"]

    def start_requests(self):
        url = 'http://www.renren.com/PLogin.do'
        # FormRequest 是Scrapy发送POST请求的方法
        yield scrapy.FormRequest(
                url = url,
                formdata = {"email" : "mr_mao_hacker@163.com", "password" : "axxxxxxxe"},
                callback = self.parse_page)

    def parse_page(self, response):
        with open("mao2.html", "w") as filename:
            filename.write(response.body)

## 标准的模拟登录

分两步:

1. 首先发送登录页面的get请求，获取到页面里的登录必须参数

2. 然后和账户密码一起post到服务器

使用FormRequest.from_response()函数，将登录页面get请求response和formdata传入函数

## 使用登录状态的Cookie模拟登录

先自己登录，获取Cookie保存至本地，然后登录是使用Cookie，虽然麻烦但是一般都能成功

cookies可以提供字典和列表类型:

1. 字典类型(name和value的键值对)

2. 列表类型(同浏览器形式一样)
cookies = [
{'name': 'Zarten', 'value': 'my name is Zarten', 'domain': 'example.com', 'path': '/currency'}
]

In [ ]:
import scrapy

class RenrenSpider(scrapy.Spider):
    name = "renren"
    allowed_domains = ["renren.com"]
    start_urls = (
        'http://www.renren.com/111111',
        'http://www.renren.com/222222',
        'http://www.renren.com/333333',
    )

    cookies = {
    "anonymid" : "ixrna3fysufnwv",
    "_r01_" : "1",
    "ap" : "327550029",
    "JSESSIONID" : "abciwg61A_RvtaRS3GjOv",
    "depovince" : "GW",
    "springskin" : "set",
    "jebe_key" : "f6fb270b-d06d-42e6-8b53-e67c3156aa7e%7Cc13c37f53bca9e1e7132d4b58ce00fa3%7C1484060607478%7C1%7C1486198628950",
    "t" : "691808127750a83d33704a565d8340ae9",
    "societyguester" : "691808127750a83d33704a565d8340ae9",
    "id" : "327550029",
    "xnsid" : "f42b25cf",
    "loginfrom" : "syshome"
    }

    # 可以重写Spider类的start_requests方法，附带Cookie值，发送POST请求
    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.FormRequest(url, cookies = self.cookies, callback = self.parse_page)


# Scrapy 中间件

中间件是介入到scrapy的Spider处理机制的钩子，主要有三个作用:

- 在Downloader生成的Response发送到Spider之前进行处理

- 在SPider生成的Request发送到Scheduler之前进行处理

- 在Spider生成的item发送到item pipeLine之前进行处理

要激活下载器中间件，将其加入到DOWNLOADER_MIDDLEWARES设置中，该设置是一个字典，键为中间件类的路径，值为其中间件的顺序

Downloader Middleware处理的过程主要在调度器发送requests请求的时候以及网页将response结果返回给spiders的时候，所以从这里我们可以知道下载中间件是介于Scrapy的request/response处理的钩子，用于修改Scrapy request和response

## 简单的代理中间件

In [ ]:
class ProxyMiddleare(object):
    logger = logging.getLogger(__name__)
    def process_request(self,request, spider):
        self.logger.debug("Using Proxy")
        request.meta['proxy'] = 'http://127.0.0.1:9743'
        return None

下载中间件的主要函数:

1. process_request(request,spider)

**当每个request通过下载中间件时，该方法被调用**，这里有一个要求，该方法必须返回以下三种中的任意一种：None,返回一个Response对象，返回一个Request对象或raise IgnoreRequest。三种返回值的作用是不同的

- None:Scrapy将继续处理该request，执行其他的中间件的相应方法，直到合适的下载器处理函数(download handler)被调用,该request被执行(其response被下载)

- Response对象：Scrapy将不会调用任何其他的process_request()或process_exception() 方法，或相应地下载函数；其将返回该response。 已安装的中间件的 process_response() 方法则会在每个response返回时被调用　(request不会被下载器下载，scrapy对接selenium时会应用到)

- Request对象：Scrapy则停止调用 process_request方法并重新调度返回的request。当新返回的request被执行后， 相应地中间件链将会根据下载的response被调用

- raise一个IgnoreRequest异常：则安装的下载中间件的 process_exception() 方法会被调用。如果没有任何一个方法处理该异常， 则request的errback(Request.errback)方法会被调用。如果没有代码处理抛出的异常， 则该异常被忽略且不记录


2. process_response(request, response, spider)

处理网页爬取过程中解决网页重定向的问题

- 如果其返回一个Response(可以与传入的response相同，也可以是全新的对象)， 该response会被在链中的其他中间件的 process_response() 方法处理

- 如果其返回一个 Request 对象，则中间件链停止， 返回的request会被重新调度下载。处理类似于 process_request() 返回request所做的那样

- 如果其抛出一个 IgnoreRequest 异常，则调用request的errback(Request.errback)。 如果没有代码处理抛出的异常，则该异常被忽略且不记录(不同于其他异常那样)


3. process_exception
在默认情况下，一次请求失败了，Scrapy会立刻原地重试，再失败再重试，如此3次。如果3次都失败了，就放弃这个请求。这种机制存在一些缺陷，以代理IP为例，若某个IP失败，这种情况下不需要scrapy机制进行重试，可以使用process_exception()捕获异常，立即更换IP


当下载处理器(download handler)或 process_request() (下载中间件)抛出异常(包括 IgnoreRequest 异常)时，Scrapy调用 process_exception()

- 如果其返回 None ，Scrapy将会继续处理该异常，接着调用已安装的其他中间件的 process_exception() 方法，直到所有中间件都被调用完毕，则调用默认的异常处理

- 如果其返回一个 Response 对象，则已安装的中间件链的 process_response() 方法被调用。Scrapy将不会调用任何其他中间件的 process_exception() 方法

- 如果其返回一个 Request 对象， 则返回的request将会被重新调用下载。这将停止中间件的 process_exception() 方法执行，就如返回一个response的那样。 这个是非常有用的，就相当于如果我们失败了可以在这里进行一次失败的重试

Scrapy提供了许多Downloader Middlewares比如负责失败重试、自动重定向等功能。他们被Downloader_Middlewares_Base变量所定义

## 设置User-Agent 

修改请求时User-Agent有几种方式:

1. 修改settings里面的USER_AGENT变量(对所有爬虫所有请求生效)

2. custom_settings = {'USER_AGENT':''}　(单个爬虫所有请求有效)

3. Request(headers={'User-Agent':''}) (单个请求有效)

4. 自定义中间件，设置随机User-Agent

In [ ]:
class ScrapySpider(scrapy.Spider):
    name = "scrapy_spider"
    allowed_domains = ["httpbin.org"]
    # 新添加的代码
    custom_settings = {
        "USER_AGENT": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
    }
    # -----------
    start_urls = (
        "https://httpbin.org/get?show_env=1",
    )

In [ ]:
import random

class RandomUserAgentMiddleware():
　　def __init__(self):
　　　　self.user_agent = [
　　　　　　'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
　　　　　　'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16',
　　　　'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
　　　　]

　　def process_request(self,request,spider):
　　　　request.headers['User-Agent'] = random.choice(self.user_agent)

## 请求重试

有三种情况决定是否进行请求重试:


可以得到response时决定是否重试

1. 判断request中是否设置了'dont_retry'参数，若没有设置则不去重试

2. 将response状态码和settings文件中的默认重试状态码进行匹配，若有匹配到的则进行请求重试

无法得到response,抛出异常时决定是否重试

3. 当无法请求时，可以根据捕获的异常来决定是否重试

In [ ]:
# scrapy 请求重试中间件部分源代码

def process_response(self, request, response, spider):
    if request.meta.get('dont_retry', False):
        return response
    if response.status in self.retry_http_codes:
        reason = response_status_message(response.status)
        return self._retry(request, reason, spider) or response
    return response

def process_exception(self, request, exception, spider):
    if isinstance(exception, self.EXCEPTIONS_TO_RETRY) \
            and not request.meta.get('dont_retry', False):
        return self._retry(request, exception, spider)

def _retry(self, request, reason, spider):
    retries = request.meta.get('retry_times', 0) + 1

    retry_times = self.max_retry_times

    if 'max_retry_times' in request.meta:
        retry_times = request.meta['max_retry_times']

    stats = spider.crawler.stats
    if retries <= retry_times:
        logger.debug("Retrying %(request)s (failed %(retries)d times): %(reason)s",
                     {'request': request, 'retries': retries, 'reason': reason},
                     extra={'spider': spider})
        retryreq = request.copy()
        retryreq.meta['retry_times'] = retries
        retryreq.dont_filter = True
        retryreq.priority = request.priority + self.priority_adjust

        if isinstance(reason, Exception):
            reason = global_object_name(reason.__class__)

        stats.inc_value('retry/count')
        stats.inc_value('retry/reason_count/%s' % reason)
        return retryreq
    else:
        stats.inc_value('retry/max_reached')
        logger.debug("Gave up retrying %(request)s (failed %(retries)d times): %(reason)s",
                     {'request': request, 'retries': retries, 'reason': reason},
                     extra={'spider': spider})


# 反反爬虫相关机制

通常防止反爬虫主要有以下几个策略

1. 动态设置User-Agent(随机切换User-Agent,模拟不同用户的浏览器信息)

2. 禁用Cookies(也就是不启用cookies middleware,不向server发送cookies,有些网站通过cookie的使用发现爬虫行为)
    可以通过 Cookies_enabled控制CookiesMiddleware开启或关闭
    
3. 设置延迟下载(防止访问过于频繁，设置为2秒或更高)，设置DOWNLOAD_DELAY

4. Google Cache或Baidu Cache:使用谷歌/百度等搜索引擎服务器页面缓存获取页面数据

5. 使用 IP地址池: VPN和代理IP

6. 使用　Crawler()

# Settings

Scrapy settings 的设计目的就是允许你通过设置来自定义所有 Scrapy 组件的行为，包括 core、extensions、pipeline 以及 spiders 自身

## 配置settings

填充 settings 的方式有多种，

1. Command line options (most precedence)
2. Settings per-spider
3. Project settings module
4. Default settings per-command
5. Default global settings (less precedence)

上面的配置的优先级是从上至下的

### 命令行模式

在运行spider时，通过 -s参数设定settings某个属性,优先级最高

In [ ]:
scrapy crawl myspider -s LOG_FILE=scrapy.log

### 单个Spiders设置

在自定义spider类中，可以使用custom_setting属性设定settings,这个设置方法优先级高于整个工程文件(settings.py)中的设置

In [ ]:
class MySpider(scrapy.Spider):
    name = 'myspider'

    custom_settings = {
        'SOME_SETTING': 'some value',
    }

### 工程项目设置

通过设定settings.py文件

### 单个命令默认配置

暂时没有体会

### 默认全局配置

全局配置位于 scrapy.settings.default_settings模块

## 访问settings

在Spider中可以使用 self.settings获取settings

settings 对象是在 spider 实例进行初始化以后在 base Spider 中设置的；如果你想要在实例化之前就想要访问相关的 settings，那么你必须覆盖 from_crawler() 方法

In [ ]:
class MySpider(scrapy.Spider):
    name = 'myspider'
    start_urls = ['http://example.com']

    def parse(self, response):
        print("Existing settings: %s" % self.settings.attributes.keys())


Settings 可以在 extensions，middleware 和 item pipelines 中通过方法 from_crawler 中的参数对象 Crawler 进行访问

In [ ]:
class MyExtension(object):
    def __init__(self, log_is_enabled=False):
        if log_is_enabled:
            print("log is enabled!")

    @classmethod
    def from_crawler(cls, crawler):
        settings = crawler.settings
        return cls(settings.getbool('LOG_ENABLED'))

## CONCURRENT_ITEMS

Scrapy downloader 并发请求(concurrent requests)的最大值

默认为16

## USER_AGENT

爬取的默认User-Agent，除非被覆盖

# Scrapy输出日志

## logging模块

使用logging模块

logging模块是Python内置的标准模块，主要用于输出运行日志，可以设置输出日志的等级、日志保存路径、日志文件回滚等

我们把python代码放入到生产环境中的时候，我们只能看到代码运行的结果，我们不知道的是代码每一步过程的最终运行状态

如果代码中间过程出现了问题的话，logging库的引用得出的日志记录可以帮助我们排查程序运行错误步骤的。方便我们修复代码，快速排查问题

print将所有信息都输出到标准输出中，严重影响开发者从标准输出中查看其它数据；logging则可以由开发者决定将信息输出到什么地方，以及怎么输出

In [1]:
import logging
logging.warning("This is a warning")

In [2]:
#引入了 logging 模块
import logging

logging.basicConfig(level = logging.INFO,format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#声明了一个 Logger 对象
logger = logging.getLogger(__name__)

logger.info("Start print log")
logger.debug("Do something")
logger.warning("Something maybe fail.")
logger.info("Finish")

### logging层级结构

![title](logging.png)

如果不创建logger实例， 直接调用logging.debug()、logging.info()logging.warning()、logging.error()、logging.critical()这些函数，那么使用的logger就是 root logger， 它可以自动创建，也是单实例的。通过logging.getLogger()或者logging.getLogger("")得到root logger实例

### logging的等级

logging有5种不同log级别:

1. DEBUG:详细信息，为了诊断问题,10

2. INFO: 确认一切正常,证明事情按预期工作,20

3. WARNING: 一些不希望发生的，但是仍可以正常执行,30

4. ERROR: 较严重的问题，不能执行某些函数,40

5. CRITICAL: 严重的问题，程序不能继续运行,50

### 使用logging

原则上，使用logging,首先你需要使用logging.basicConfig()设定一个基础配置,实际上这个配置是可选的

如果不进行这个配置，那么默认的Logger是root,默认的basicConfig level是WARNING,意味着只有来自与WARNING级别或者更高级别的才会被记录

In [8]:
import logging
logging.basicConfig(level=logging.INFO)

def hypotenuse(a, b):
    """Compute the hypotenuse"""
    return (a**2 + b**2)**0.5


logging.info("Hypotenuse of {a}, {b} is {c}".format(a=3, b=4, c=hypotenuse(a,b)))

TypeError: unsupported operand type(s) for ** or pow(): 'type' and 'int'

log输出消息格式:{LEVEL}:{LOGGER}:{MESSAGE}

默认LOGGER是root

不同的LOGGER可以记录不同类型和格式的消息，比如你可以配置一个logger用于输出到终端，另一个logger将日志保存至文件对于一个特定的模块可以使用特定的log level

In [9]:
import logging
logging.basicConfig(level=logging.WARNING)

def hypotenuse(a, b):
    """Compute the hypotenuse"""
    return (a**2 + b**2)**0.5

kwargs = {'a':3, 'b':4, 'c':hypotenuse(3, 4)}

logging.debug("a = {a}, b = {b}".format(**kwargs))
logging.info("Hypotenuse of {a}, {b} is {c}".format(**kwargs))
logging.warning("a={a} and b={b} are equal".format(**kwargs))
logging.error("a={a} and b={b} cannot be negative".format(**kwargs))
logging.critical("Hypotenuse of {a}, {b} is {c}".format(**kwargs))

ERROR:root:a=3 and b=4 cannot be negative
CRITICAL:root:Hypotenuse of 3, 4 is 5.0


### log输出至文件

默认日志消息在终端输出，如果想保存至文件中，可以在logging.basicConfig()函数设置file参数

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, file='sample.log')

### 改变输出格式

前面已经说了默认格式为: {LEVEL}:{LOGGER}:{MESSAGE}

若想自定义格式，可以在logging.basicConfig()函数设置format参数

In [1]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s :: %(levelname)s :: %(message)s')
logging.info("Just like that!")

2020-02-19 22:11:42,901 :: INFO :: Just like that!


### 创建新的 logger

在所有模块中使用root logger不是一个好习惯，因为它们共享同一个root logger，一旦root logger进行了初始配置那么就不能再更改，比如你想在myprojectmodule中输出日志至一个文件，然后在main module中输出日志至另一文件，使用root是不可能实现的

In [ ]:
# 1. code inside myprojectmodule.py
import logging
logging.basicConfig(file='module.log')

#-----------------------------

# 2. code inside main.py (imports the code from myprojectmodule.py)
import logging
import myprojectmodule  # This runs the code in myprojectmodule.py

logging.basicConfig(file='main.log')  # No effect, because!

使用logger.getLogger(name)的方式创建一个新的logger,如果使用相同的name,那么会替换之前的logging

习惯上使用__name__变量传递给getLogger

In [2]:
import logging
logger = logging.getLogger(__name__)
logger.info('my logging message')

使用__name__的好处是避免了硬编码,因为__name__记录了模块(python文件)的名字

一旦创建了新的logger，那么后面你的日志输出要使用logger.info()而不能再使用logging.info()

还有要注意所有创建的logger会继承root logger的某些配置，比如你在root logger中设定日志输出至文件，然而你自定义的logger没有配置日志输出方式，那么其将继承root logger，于是自定义的logger也会将日志输出至root logger配置的文件中

### 设置File Handler和Formatter

FileHandler()和Formatter()用于非root logger配置其输出的文件以及消息格式

它继承了StreamHandler的输出功能

In [4]:
import logging

# Gets or creates a logger
logger = logging.getLogger(__name__)  

# set log level
logger.setLevel(logging.WARNING)

# define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

# Logs
logger.debug('A debug message')
logger.info('An info message')
logger.warning('Something is not right.')
logger.error('A Major error has happened.')
logger.critical('Fatal error. Cannot continue')

### StreamHandler

位于核心日志记录包中的StreamHandler类将日志记录输出发送到诸如sys.stdout，sys.stderr或任何类似文件的对象

### 日志记录traceback信息

除了记录debug info warning error critical消息，你也可以使用 logger.exception 记录traceback消息

In [1]:
import logging

# Create or get the logger
logger = logging.getLogger(__name__)  

# set log level
logger.setLevel(logging.INFO)

def divide(x, y):
    try:
        out = x / y
    except ZeroDivisionError:
        logger.exception("Division by zero problem")
    else:
        return out

# Logs
logger.error("Divide {x} / {y} = {c}".format(x=10, y=0, c=divide(10,0)))

Division by zero problem
Traceback (most recent call last):
  File "<ipython-input-1-275b4d096208>", line 11, in divide
    out = x / y
ZeroDivisionError: division by zero
Divide 10 / 0 = None


### 日志同时输出到终端和写入文件

一个logger可以指定多个handler，多个handler可以设计成不同的初始级别和输出方式

In [ ]:
# 第一步，创建一个logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)    # Log等级总开关
 
# 第二步，创建一个handler，用于写入日志文件
logfile = './log/logger.txt'
fh = logging.FileHandler(logfile, mode='w')
fh.setLevel(logging.DEBUG)   # 输出到file的log等级的开关
 
# 第三步，再创建一个handler，用于输出到控制台
ch = logging.StreamHandler() # 默认是输出到终端
ch.setLevel(logging.WARNING)   # 输出到console的log等级的开关
 
# 第四步，定义handler的输出格式
formatter = logging.Formatter("%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s")
fh.setFormatter(formatter)
ch.setFormatter(formatter)
 
# 第五步，将logger添加到handler里面
logger.addHandler(fh)
logger.addHandler(ch)

## Spider log

Scrapy为每个Spider实例提供了一个logger

In [ ]:
import scrapy

class MySpider(scrapy.Spider):

    name = 'myspider'
    start_urls = ['http://scrapinghub.com']

    def parse(self, response):
        self.logger.info('Parse function called on %s', response.url)

### scrapy logging 配置

Logging设置

以下设置可以被用来配置logging:
- LOG_ENABLED
默认: True，启用logging

- LOG_ENCODING
默认: 'utf-8'，logging使用的编码

- LOG_FILE
默认: None，logging输出的文件名

- LOG_LEVEL
默认: 'DEBUG'，log的最低级别

- LOG_STDOUT
默认: False
如果为 True，进程所有的标准输出(及错误)将会被重定向到log中。例如，执行 print 'hello' ，其将会在Scrapy log中显示。

## scrapy性能提升配置

### 增加并发

并发是指同时处理的request的数量，scrapy默认32

Scrapy默认的全局并发限制对同时爬取大量网站的情况并不适用，因此您需要增加这个值。 增加多少取决于您的爬虫能占用多少CPU。 一般开始可以设置为 100

In [ ]:
CONCURRENT_REQUESTS = 100

### 降级log级别

当进行通用爬取时，一般您所注意的仅仅是爬取的速率以及遇到的错误。 Scrapy使用 INFO log级别来报告这些信息。为了减少CPU使用率(及记录log存储的要求), 在生产环境中进行通用爬取时您不应该使用 DEBUG log级别。 不过在开发的时候使用 DEBUG 应该还能接受

In [ ]:
LOG_LEVEL = 'INFO'

### 禁止cookies

除非您 真的 需要，否则请禁止cookies。在进行通用爬取时cookies并不需要， (搜索引擎则忽略cookies)。禁止cookies能减少CPU使用率及Scrapy爬虫在内存中记录的踪迹，提高性能

In [ ]:
COOKIES_ENABLED = False

### 禁止重试

对失败的HTTP请求进行重试会减慢爬取的效率，尤其是当站点响应很慢(甚至失败)时， 访问这样的站点会造成超时并重试多次。这是不必要的，同时也占用了爬虫爬取其他站点的能力

In [ ]:
RETRY_ENABLED = False

### 减少下载超时

如果您对一个非常慢的连接进行爬取(一般对通用爬虫来说并不重要)， 减小下载超时能让卡住的连接能被快速的放弃并解放处理其他站点的能力

In [ ]:
DOWNLOAD_TIMEOUT = 15

### 禁止重定向

除非您对跟进重定向感兴趣，否则请考虑关闭重定向。 当进行通用爬取时，一般的做法是保存重定向的地址，并在之后的爬取进行解析。 这保证了每批爬取的request数目在一定的数量， 否则重定向循环可能会导致爬虫在某个站点耗费过多资源

In [ ]:
REDIRECT_ENABLED = False

### 使用ip代理池

### 使用user_agent池

###  设置延迟

延迟即当一个请求完成时休息一会再发送第二个请求，DOWMLOAD_DELY=3,设置延迟下载可以避免被发现

### 暂停和恢复爬虫

有一个方法可以暂时的存储你爬的状态，当爬虫中断的时候继续打开后依然可以从中断的地方爬

只需要在setting.py中JOB_DIR=file_name 其中填的是你的文件目录，注意这里的目录不允许共享，只能存储单独的一个spdire的运行状态，如果你不想在从中断的地方开始运行，只需要将这个文件夹删除即可

当然还有其他的放法：scrapy crawl somespider -s JOBDIR=crawls/somespider-1，这个是在终端启动爬虫的时候调用的，可以通过ctr+c中断，恢复还是输入上面的命令

### 不按照robots.txt

ROBOTSTXT_OBEY = False

### 配置请求头

## Scrapy提早结束抓取

Scrapy的CloseSpider扩展可以在条件达成时，自动结束抓取。你可以用CLOSESPIDER_TIMEOUT(in seconds)， CLOSESPIDER_ITEMCOUNT， CLOSESPIDER_PAGECOUNT，和CLOSESPIDER_ERRORCOUNT分别设置在一段时间、抓取一定数量的文件、发出一定数量请求、发生一定数量错误时，提前关闭爬虫

In [ ]:
scrapy crawl fast -s CLOSESPIDER_ITEMCOUNT=10
scrapy crawl fast -s CLOSESPIDER_PAGECOUNT=10
scrapy crawl fast -s CLOSESPIDER_TIMEOUT=10

# 爬虫搜索策略

在爬虫系统中，待抓取URL队列是很重要的部分。待抓取URL队列中的URL以什么样的顺序排列也是一个很重要的问题，因为这涉及到先抓取哪个页面，后抓取哪个页面。而决定URL排列顺序的方法，叫做抓取策略

## 深度优先策略(Depth-First Search)

即图的深度优先遍历算法。网络爬虫会先从起始页开始，一个链接一个链接跟踪下去，处理完这条线路之后再转入下一个起始页，继续跟踪链接

![title](深度优先.png)

## 广度优先策略(Breadth First Search)

广度优先基本思路是，将新下载网页中发现的链接直接插入待抓取URL队列的末尾。即网络爬虫会选抓取起始网页中链接的所有网页，然后在选择其中一个链接网页，继续抓取在此网页中的链接的所有网页

有很多研究将广度优先搜索策略应用于聚焦爬虫中。其基本思想是认为与初始URL在一定链接距离内的网页具有主题相关性的概率很大

![title](广度优先.png)

## 广度优先搜索和深度优先搜索

深度优先搜索算法涉及的是堆栈

广度优先搜索涉及的是队列

堆栈(stacks)具有后进先出(last in first out，LIFO)的特征

队列(queue)是一种具有先进先出(first in first out，FIFO)特征的线性数据结构

## Scrapy的搜索策略

默认情况下，Scrapy使用LIFO队列来存储等待的请求，即深度优先

在Settings文件中，可以设置爬虫允许的最大深度

DEPTH_LIMIT=3,可以通过meta查看当前深度，0表示无深度

爬取时，DEPTH_PRIORITY=0表示深度优先(默认),1表示广度优先

DEPTH_PRIORITY = 0
SCHEDULER_DISK_QUEUE = 'scrapy.squeues.PickleLifoDiskQueue'
SCHEDULER_MEMORY_QUEUE = 'scrapy.squeues.LifoMemoryQueue'

如果你想以广度优先顺序进行爬取，进行如下设置

In [ ]:
DEPTH_PRIORITY = 1
SCHEDULER_DISK_QUEUE = 'scrapy.squeue.PickleFifoDiskQueue'
SCHEDULER_MEMORY_QUEUE = 'scrapy.squeue.FifoMemoryQueue'

# CrawlSpider

它是Spider的派生类，Spider的设计原则是只爬取start_url列表中的网页，而从爬取的网页中获取link并继续爬取的工作CrawlSpider类更适合

In [ ]:
# -t crawl表示创建CrawlSpider spider
scrapy genspider -t crawl spiderName 地址

它与Spider类的最大不同是多了一个rules函数，其作用是定义提取动作,在rules中包含一个或多个Rule,每一个rule就是批量网页请求。一个Rules中多个rule执行顺序是从上到下

In [ ]:
rules = (
    Rule(LinkExtractor(allow=r'/web/site0/tab5240/info\d+\.htm'), callback='parse_item'),
    Rule(LinkExtractor(allow=r'http://bxjg.circ.gov.cn/web/site0/tab5240/module14430/page\d\.htm'),follow=True),

)

In [ ]:
class scrapy.contrib.spiders.Rule (
link_extractor, 
    callback=None,
    cb_kwargs=None,
    follow=None,
    process_links=None,
    process_request=None )

callback 指定获取链接后的回调函数

follow：指定了根据该规则从 response提取的连结是否需要跟进。
        当 callback为 None,预设值为 true
    
process_links：主要用来过滤由 link_extractor获取到的链接

process_request：主要用来过滤在 rule中提取到的 request

避免使用parse()作为回调函数

由于CrawlSpider 使用 parse( )方法来实现其逻辑，如果 parse()方法覆盖了，CrawlSpider 将会运行失败CrawlSpider 继承自 Spider 类。 

Spider 类的所有方法和属性都可以使用（比如重写start_requests( )）

# 分布式Scrapy爬取

爬虫基本流程

![title](web_crawl_procedure.png)

Scrapy_redis改变了scrapy的队列调度，将起始的网址从start_urls里分离出来，改为从redis读取，多个客户端可以同时读取同一个redis,从而实现了分布式的爬虫

![title](scrapy_redis.png)

## Scrapy_redis组件

scrapy-redis在scrapy的架构上增加了redis，基于redis的特性拓展了如下四种组件：Scheduler，Duplication Filter，Item Pipeline，Base Spider

### Scheduler

scrapy改造了python本来的collection.deque(双向队列)形成了自己的Scrapy queue，但是Scrapy多个spider不能共享待爬取队列Scrapy queue，即Scrapy本身不支持爬虫分布式，scrapy-redis 的解决是把这个Scrapy queue换成redis数据库（也是指redis队列），从同一个redis-server存放要爬取的request，便能让多个spider去同一个数据库里读取。

Scrapy中跟“待爬队列”直接相关的就是调度器Scheduler，它负责对新的request进行入列操作（加入Scrapy queue），取出下一个要爬取的request（从Scrapy queue中取出）等操作。它把待爬队列按照优先级建立了一个字典结构，然后根据request中的优先级，来决定该入哪个队列，出列时则按优先级较小的优先出列。

为了管理这个比较高级的队列字典，Scheduler需要提供一系列的方法。但是原来的Scheduler已经无法使用，所以使用Scrapy-redis的scheduler组件

### Duplication Filter

Scrapy中用集合实现这个request去重功能，Scrapy中把已经发送的request指纹放入到一个集合中，把下一个request的指纹拿到集合中比对，如果该指纹存在于集合中，说明这个request发送过了，如果没有则继续操作

在scrapy-redis中去重是由Duplication Filter组件来实现的，它通过redis的set不重复的特性，巧妙的实现了DuplicationFilter去重。

scrapy-redis调度器从引擎接受request，将request的指纹存入redis的set检查是否重复，并将不重复的request push写入redis的 request queue。

引擎请求request(Spider发出的）时，调度器从redis的request queue队列里根据优先级pop 出⼀个request 返回给引擎，引擎将此request发给spider处理

###  Item Pipeline

引擎将(Spider返回的)爬取到的Item给Item Pipeline，scrapy-redis 的Item Pipeline将爬取到的 Item 存入redis的 items queue。

修改过Item Pipeline可以很方便的根据 key 从 items queue
提取item，从而实现 items processes集群

### Base Spider

不在使用scrapy原有的Spider类，重写的RedisSpider继承了Spider和RedisMixin这两个类，RedisMixin是用来从redis读取url的类

当我们生成一个Spider继承RedisSpider时，调用setup_redis函数，这个函数会去连接redis数据库，然后会设置signals(信号)：一个是当spider空闲时候的signal，会调用spider_idle函数，这个函数调用schedule_next_request函数，保证spider是一直活着的状态，并且抛出DontCloseSpider异常。一个是当抓到一个item时signal，会调用item_scraped函数，这个函数会调 schedule_next_request函数，获取下一个request

总结一下scrapy-redis的总体思路：这套组件通过重写scheduler和spider类，实现了调度、spider启动和redis的交互

实现新的dupefilter和queue类，达到了判重和调度容器和redis的交互，因为每个主机上的爬虫进程都访问同一个redis数据库，所以调度和判重都统一进行统一管理，达到了分布式爬虫的目的；当spider被初始化时，同时会初始化一个对应的scheduler对象，这个调度器对象通过读取settings，配置好自己的调度容器queue和判重工具dupefilter；

每当一个spider产出一个request的时候，scrapy引擎会把这个reuqest递交给这个spider对应的scheduler对象进行调度，scheduler对象通过访问redis对request进行判重，如果不重复就把他添加进redis中的调度器队列里。当调度条件满足时，scheduler对象就从redis的调度器队列中取出一个request发送给spider，让他爬取；

当spider爬取的所有暂时可用url之后，scheduler发现这个spider对应的redis的调度器队列空了，于是触发信号spider_idle，spider收到这个信号之后，直接连接redis读取strart_url池，拿去新的一批url入口，然后再次重复上边的工作

In [28]:
class cl1:
    def __init__(self,num1,num2):
        self.num1 = num1
        self.num2 = num2
        self.jian = self.sub(num1,num2)
        self.sum = self.add()
    def add(self):
        return self.num1+self.num2
    

class cl2:
    def sub(self,num1,num2):
        return self.num1 - self.num2
    
class cl3(cl1,cl2):
    def __init__(self,num1,num2):
        super(cl3,self).__init__(num1,num2)
        
    
obj = cl3(1,2)


AttributeError: 'cl3' object has no attribute 'mro'